# Lab 3.2.4: GPTQ Quantization - Solutions

This notebook contains solutions for all exercises in Lab 3.2.4.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

## Exercise 1 Solution: GPTQ Algorithm Implementation

Implement the core GPTQ algorithm from scratch.

In [ ]:
def gptq_quantize_layer(
    weight: np.ndarray,
    hessian: np.ndarray,
    bits: int = 4,
    group_size: int = 128,
    block_size: int = 128,
    percdamp: float = 0.01
) -> tuple:
    """
    GPTQ quantization for a single layer.
    
    Args:
        weight: Weight matrix (out_features, in_features)
        hessian: Hessian matrix (in_features, in_features)
        bits: Quantization bits
        group_size: Group size for quantization
        block_size: Block size for processing
        percdamp: Dampening factor
        
    Returns:
        Tuple of (quantized_weight, scales, zeros)
    """
    out_features, in_features = weight.shape
    
    # Clone weight for modification
    W = weight.copy()
    
    # Add dampening to Hessian diagonal
    H = hessian.copy()
    damp = percdamp * np.mean(np.diag(H))
    H += damp * np.eye(in_features)
    
    # Cholesky decomposition
    try:
        L = np.linalg.cholesky(H)
        H_inv = np.linalg.inv(L.T) @ np.linalg.inv(L)
    except:
        # Fallback if Cholesky fails
        H_inv = np.linalg.pinv(H)
    
    # Quantization parameters
    qmax = 2 ** bits - 1
    num_groups = (in_features + group_size - 1) // group_size
    
    # Output arrays
    Q = np.zeros_like(W, dtype=np.int8)
    scales = np.zeros((out_features, num_groups), dtype=np.float32)
    zeros = np.zeros((out_features, num_groups), dtype=np.float32)
    
    # Process in blocks
    for i in range(0, in_features, block_size):
        i_end = min(i + block_size, in_features)
        
        # Block of weights
        W_block = W[:, i:i_end]
        H_block_inv = H_inv[i:i_end, i:i_end]
        
        # Quantize block column by column
        for j in range(i_end - i):
            col_idx = i + j
            group_idx = col_idx // group_size
            
            # Compute scale for this group (if first in group)
            if col_idx % group_size == 0:
                group_end = min(col_idx + group_size, in_features)
                group_weights = W[:, col_idx:group_end]
                
                w_min = group_weights.min(axis=1)
                w_max = group_weights.max(axis=1)
                
                scales[:, group_idx] = (w_max - w_min) / qmax
                zeros[:, group_idx] = w_min
            
            # Get current column
            w = W[:, col_idx]
            scale = scales[:, group_idx]
            zero = zeros[:, group_idx]
            
            # Quantize
            q = np.clip(np.round((w - zero) / (scale + 1e-10)), 0, qmax).astype(np.int8)
            Q[:, col_idx] = q
            
            # Dequantize
            w_quant = q * scale + zero
            
            # Compute error
            error = w - w_quant
            
            # Apply GPTQ correction to remaining columns
            if col_idx + 1 < in_features and j < i_end - i - 1:
                h_jj_inv = H_block_inv[j, j]
                correction = np.outer(error, H_block_inv[j, j+1:] / (h_jj_inv + 1e-10))
                W[:, col_idx+1:i_end] -= correction[:, :i_end-col_idx-1]
    
    return Q, scales, zeros


# Test GPTQ implementation
np.random.seed(42)

# Simulate weight matrix
weight = np.random.randn(256, 512).astype(np.float32) * 0.02

# Simulate Hessian from calibration data
calibration_data = np.random.randn(100, 512).astype(np.float32)
hessian = (calibration_data.T @ calibration_data) / 100

print("GPTQ Quantization Test")
print("="*50)

# Quantize
Q, scales, zeros = gptq_quantize_layer(weight, hessian, bits=4, group_size=128)

# Dequantize for evaluation
num_groups = scales.shape[1]
W_dequant = np.zeros_like(weight)
for g in range(num_groups):
    start = g * 128
    end = min((g + 1) * 128, 512)
    W_dequant[:, start:end] = Q[:, start:end] * scales[:, g:g+1] + zeros[:, g:g+1]

# Compute error
mse = np.mean((weight - W_dequant) ** 2)
print(f"MSE: {mse:.8f}")
print(f"Max error: {np.max(np.abs(weight - W_dequant)):.6f}")

# Compare with naive quantization
scale_naive = (weight.max() - weight.min()) / 15
q_naive = np.clip(np.round((weight - weight.min()) / scale_naive), 0, 15)
w_naive = q_naive * scale_naive + weight.min()
mse_naive = np.mean((weight - w_naive) ** 2)

print(f"\nNaive quantization MSE: {mse_naive:.8f}")
print(f"GPTQ improvement: {mse_naive/mse:.2f}x lower error")

## Exercise 2 Solution: Group Size Impact Analysis

Analyze how group size affects quality and memory.

In [ ]:
def analyze_group_size_impact(weight: np.ndarray, group_sizes: list) -> dict:
    """
    Analyze impact of different group sizes on GPTQ quality.
    
    Args:
        weight: Weight matrix
        group_sizes: List of group sizes to test
        
    Returns:
        Analysis results
    """
    results = {}
    
    # Create synthetic Hessian
    in_features = weight.shape[1]
    calibration = np.random.randn(100, in_features).astype(np.float32)
    hessian = (calibration.T @ calibration) / 100
    
    for group_size in group_sizes:
        # Quantize
        Q, scales, zeros = gptq_quantize_layer(
            weight, hessian, bits=4, group_size=group_size
        )
        
        # Dequantize
        num_groups = scales.shape[1]
        W_dequant = np.zeros_like(weight)
        for g in range(num_groups):
            start = g * group_size
            end = min((g + 1) * group_size, in_features)
            W_dequant[:, start:end] = Q[:, start:end] * scales[:, g:g+1] + zeros[:, g:g+1]
        
        # Metrics
        mse = np.mean((weight - W_dequant) ** 2)
        
        # Memory calculation
        weight_bits = weight.size * 4  # 4-bit weights
        scale_bits = scales.size * 16  # FP16 scales
        zero_bits = zeros.size * 16    # FP16 zeros
        total_bits = weight_bits + scale_bits + zero_bits
        effective_bits = total_bits / weight.size
        
        results[group_size] = {
            'mse': mse,
            'effective_bits': effective_bits,
            'compression': 32 / effective_bits,
            'num_groups': num_groups
        }
    
    return results


# Test different group sizes
np.random.seed(42)
weight = np.random.randn(4096, 4096).astype(np.float32) * 0.02

group_sizes = [32, 64, 128, 256, 512, 1024]
results = analyze_group_size_impact(weight, group_sizes)

print("Group Size Impact Analysis")
print("="*60)
print(f"{'Group Size':<12} {'MSE':<15} {'Eff. Bits':<12} {'Compression':<12}")
print("-"*60)

for gs in group_sizes:
    r = results[gs]
    print(f"{gs:<12} {r['mse']:<15.8f} {r['effective_bits']:<12.2f} {r['compression']:<12.1f}x")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

mses = [results[gs]['mse'] for gs in group_sizes]
compressions = [results[gs]['compression'] for gs in group_sizes]

axes[0].semilogx(group_sizes, mses, 'bo-', base=2)
axes[0].set_xlabel('Group Size')
axes[0].set_ylabel('MSE')
axes[0].set_title('Quantization Error vs Group Size')
axes[0].grid(True)

axes[1].semilogx(group_sizes, compressions, 'ro-', base=2)
axes[1].set_xlabel('Group Size')
axes[1].set_ylabel('Compression Ratio')
axes[1].set_title('Compression vs Group Size')
axes[1].grid(True)

plt.tight_layout()
plt.show()

print("\nRecommendation: Group size 128 provides good quality/compression balance")

## Exercise 3 Solution: Calibration Data Sensitivity

Analyze how calibration data affects GPTQ quality.

In [ ]:
def analyze_calibration_sensitivity(
    weight: np.ndarray,
    calibration_sizes: list,
    num_trials: int = 5
) -> dict:
    """
    Analyze how calibration data size affects GPTQ quality.
    
    Args:
        weight: Weight matrix
        calibration_sizes: List of calibration sizes to test
        num_trials: Number of trials per size
        
    Returns:
        Analysis results with mean and std
    """
    in_features = weight.shape[1]
    results = {}
    
    for cal_size in calibration_sizes:
        mses = []
        
        for trial in range(num_trials):
            # Generate calibration data
            calibration = np.random.randn(cal_size, in_features).astype(np.float32)
            hessian = (calibration.T @ calibration) / cal_size
            
            # Quantize
            Q, scales, zeros = gptq_quantize_layer(
                weight, hessian, bits=4, group_size=128
            )
            
            # Dequantize and compute error
            num_groups = scales.shape[1]
            W_dequant = np.zeros_like(weight)
            for g in range(num_groups):
                start = g * 128
                end = min((g + 1) * 128, in_features)
                W_dequant[:, start:end] = Q[:, start:end] * scales[:, g:g+1] + zeros[:, g:g+1]
            
            mse = np.mean((weight - W_dequant) ** 2)
            mses.append(mse)
        
        results[cal_size] = {
            'mean_mse': np.mean(mses),
            'std_mse': np.std(mses),
            'min_mse': np.min(mses),
            'max_mse': np.max(mses)
        }
    
    return results


# Test calibration sensitivity
np.random.seed(42)
weight = np.random.randn(512, 512).astype(np.float32) * 0.02

calibration_sizes = [10, 25, 50, 100, 200, 500, 1000]
results = analyze_calibration_sensitivity(weight, calibration_sizes, num_trials=5)

print("Calibration Data Sensitivity Analysis")
print("="*60)
print(f"{'Cal Size':<10} {'Mean MSE':<15} {'Std MSE':<15} {'Variability':<12}")
print("-"*60)

for size in calibration_sizes:
    r = results[size]
    variability = r['std_mse'] / r['mean_mse'] * 100
    print(f"{size:<10} {r['mean_mse']:<15.8f} {r['std_mse']:<15.8f} {variability:<12.1f}%")

# Visualization
plt.figure(figsize=(10, 5))

means = [results[s]['mean_mse'] for s in calibration_sizes]
stds = [results[s]['std_mse'] for s in calibration_sizes]

plt.errorbar(calibration_sizes, means, yerr=stds, fmt='bo-', capsize=5)
plt.xlabel('Calibration Data Size')
plt.ylabel('MSE')
plt.title('GPTQ Quality vs Calibration Data Size')
plt.xscale('log')
plt.grid(True)
plt.show()

print("\nRecommendation: Use at least 100-200 calibration samples for stable results")

## Summary

Key findings:

1. **GPTQ algorithm** reduces quantization error through Hessian-based correction
2. **Group size 128** offers the best quality/compression trade-off
3. **Calibration data** of 100-200 samples provides stable results
4. **GPTQ achieves 2-5x lower error** than naive quantization